In [52]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import math

## Hyperparameters

In [53]:
epochs = 500
learning_rate = 0.001
batch_size = 64


device = torch.device("mps")

## Load the Datasets and setup the Data Loaders (with data augmentation)

In [54]:
transform_normal = transforms.Compose([
    transforms.Resize((250, 250)),
    transforms.ToTensor()
])

transform_train = transforms.Compose([
    transforms.Resize((250, 250)),
    # Randomly change the brightness of the image
    transforms.ColorJitter(brightness=0.5),
    # Randomly flip the image horizontally
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),  # Randomly flip the image vertically
    transforms.RandomRotation(55),  # Randomly rotate the image by 45 degrees
    transforms.ToTensor(),
    # transforms.Normalize(mean = [0.0, 0.0, 0.0], std = [1.0, 1.0, 1.0]) # Takes each value for the channel, subtracts the mean and divides by the standard deviation (value - mean) / std
])

# Define the transformations
transformations1 = transforms.Compose(
    [transforms.ToTensor(), transforms.Resize((250, 250))])

# Load the dataset
training_dataset = torchvision.datasets.Flowers102(root='./data', split="train",
                                                   download=True, transform=transform_train)
testing_dataset = torchvision.datasets.Flowers102(root='./data', split="test",
                                                  download=True, transform=transformations1)
validation_dataset = torchvision.datasets.Flowers102(root='./data', split="val",
                                                     download=True, transform=transformations1)

# Create the dataloaders
train_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testing_dataset, batch_size=batch_size, shuffle=False)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

## Create The Model

In [55]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.flatten = nn.Flatten()
    self.relu = nn.PReLU()
    
    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(16),
        nn.PReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(32),
        nn.PReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.layer3 = nn.Sequential(
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(64),
        nn.PReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.layer4 = nn.Sequential(
        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(128),
        nn.PReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.layer5 = nn.Sequential(
        nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(256),
        nn.PReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.layer6 = nn.Sequential(
        nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(256),
        nn.PReLU(),
        nn.MaxPool2d(2, 2)
    )
    
    # self.fc1 = nn.Linear(256 * 3 * 3, 1024)
    self.fc1 = nn.Linear(256 * 3 * 3, 102)
    self.drop = nn.Dropout(p=0.25)
    self.fc2 = nn.Linear(1024, 256)
    self.fc3 = nn.Linear(256, 102)  # Output layer for 102 classes

  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.layer5(x)
    x = self.layer6(x)
    
    x = self.flatten(x)
    x = self.fc1(x)
    # x = self.relu(x)
    # x = self.drop(x)
    # x = self.fc2(x)
    # x = self.relu(x)
    # x = self.fc3(x)
    return x


In [56]:
device = torch.device("mps")
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
# scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.3, total_iters=8)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)

## Network Accuracy Tests (Validation and Testing)

In [57]:
def NetworkAccuracyOnValidation():
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        # num_class_correct = [0 for i in range(102)]
        # num_class_samples = [0 for i in range(102)]
        total_correct = 0
        total_samples = 0
        for images, labels in validation_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

            # for i in range(len(labels)):
            #     label = labels[i]
            #     pred = predictions[i]
            #     if label == pred:
            #         num_class_correct[label] += 1
            #     num_class_samples[label] += 1

    val_epoch_loss = val_loss / val_total
    val_epoch_acc = 100. * val_correct / val_total
    
    return val_epoch_loss, val_epoch_acc


def NetworkAccuracyOnTesting():
    with torch.no_grad():
        num_class_correct = [0 for i in range(102)]
        num_class_samples = [0 for i in range(102)]
        total_correct = 0
        total_samples = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)

            _, predictions = torch.max(outputs, 1)
            total_samples += labels.size(0)
            total_correct += (predictions == labels).sum().item()

            for i in range(len(labels)):
                label = labels[i]
                pred = predictions[i]
                if (label == pred):
                    num_class_correct[label] += 1
                num_class_samples[label] += 1

        acc = 100.0 * total_correct / total_samples
        print(f'Accuracy on testing set: {acc} %')

        for i in range(102):
            acc = 100.0 * num_class_correct[i] / num_class_samples[i]
            print(f'Accuracy of {i} : {acc} %')

        return acc

## Training and Testing loop

In [58]:
best_accuracy_epoch = 0
best_accuracy = 0
early_stopping_patience = 15
no_improve_epochs = 0

for epoch in range(epochs):  # Loop over the dataset multiple times
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(train_loader, 0):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        
        label_pred = model(images)
        loss = criterion(label_pred, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = label_pred.max(1)
        # predicted = torch.max(label_pred, 1)[1]
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        batch_corr = (predicted == labels).sum()
        batch_acc = batch_corr.item() / len(images)
        
        
        # print(f"Epoch Number {epoch}, Index = {i}/{len(train_loader)-1}, Loss = {loss.item()}")
    
    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')
    
    val_epoch_loss, val_epoch_acc = NetworkAccuracyOnValidation()
    print(f'Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_epoch_acc:.2f}%')
    
    scheduler.step(val_epoch_loss)
    
    if (val_epoch_acc > best_accuracy):
        best_accuracy = val_epoch_acc
        best_accuracy_epoch = epoch
        torch.save(model.state_dict(), 'best_model.pth')
        no_improve_epochs = 0
    else:
        no_improve_epochs += 1
        if no_improve_epochs >= early_stopping_patience:
            print(f"Early stopping at epoch {epoch}")
            break
    
        

    scheduler.step()
        
print(f"Best accuracy on validation split: {best_accuracy} at epoch {best_accuracy_epoch}")
model.eval()
testing_accuracy = NetworkAccuracyOnTesting()
print(f"Testing accuracy: {testing_accuracy}")

Epoch Number 0, Loss = 4.037531852722168, Accuracy = 5.882352941176471
Epoch Number 1, Loss = 3.7695438861846924, Accuracy = 10.686274509803921
Epoch Number 2, Loss = 3.428579568862915, Accuracy = 15.686274509803921
Epoch Number 3, Loss = 2.939225912094116, Accuracy = 17.15686274509804
Epoch Number 4, Loss = 2.7528235912323, Accuracy = 22.54901960784314
Epoch Number 5, Loss = 2.676069498062134, Accuracy = 24.901960784313726
Epoch Number 6, Loss = 2.266962766647339, Accuracy = 26.176470588235293
Epoch Number 7, Loss = 2.3507659435272217, Accuracy = 24.41176470588235
Epoch Number 8, Loss = 2.233698606491089, Accuracy = 26.96078431372549
Epoch Number 9, Loss = 1.987800121307373, Accuracy = 27.941176470588236
Epoch Number 10, Loss = 2.0534770488739014, Accuracy = 29.607843137254903
Epoch Number 11, Loss = 1.9027165174484253, Accuracy = 29.11764705882353
Epoch Number 12, Loss = 2.012486457824707, Accuracy = 32.254901960784316
Epoch Number 13, Loss = 1.721361517906189, Accuracy = 33.62745098